In [1]:
import sys
# to import local fuctions
sys.path.insert(0, '../tools')
import pandas as pd
from datetime import timedelta

from run_all import main

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)

# Running Full Script

In [2]:
# If you want to run it on a small pool of stocks
#trades = main(stocks_path='../docs/my_stocks_test.csv')
# Run for all stocks
trades = main()

Start Update Prices Process ...
Running Incremental Refresh
Run Prices
Merging with Existing Prices
It took 48 seconds to extract prices.
It took 77 seconds to create PE Ratios.
Running Full Refresh of Indicators. This will take some time.
Running indicators ...
Running indicators ...
Running indicators ...
Running indicators ...
Running indicators ...
Running indicators ...
Running indicators ...
Running indicators ...
Running indicators ...
Running indicators ...
Running indicators ...
Running indicators ...
Running indicators ...
Running indicators ...
Running indicators ...
Running indicators ...
Running indicators ...
Exporting Indicators
It took 0.7666666666666667 minutes to generate indicators.
Create Buy/Sell Signals
It took 0 seconds to generate Buy and Sell Signals
The whole process took 2.066666666666667 minutes.


In [3]:
trades.head()

,timestamp,high_price,low_price,open_price,close_price,volume,symbol,interval,just_date,name,industry,eps,ttm_eps,pe_ratio,low_is_min_7,low_is_min_14,momentum_rsi,index,close_price_x,close_price_shift,high_previous,low_previous,close_previous,open_previous,year,daily_return,moving_1d_return,moving_2d_return,moving_3d_return,moving_4d_return,moving_5d_return,moving_7d_return,moving_30d_return_mean,moving_30d_return_std,moving_30d_min_return,moving_60d_return_mean,moving_60d_return_std,moving_60d_min_return,moving_90d_return_mean,moving_90d_return_std,moving_90d_min_return,sma_10d,sma_10d_shift,sma_10d_coef,sma_10d_std,stationary_sma_10d,stationary_sma_10d_zscore,stationary_sma_10d_zscore_shift,stationary_sma_10d_zscore_shift_2,stationary_sma_10d_zscore_min,stationary_sma_10d_zscore_max,sma_20d,sma_20d_shift,sma_20d_coef,sma_20d_std,stationary_sma_20d,stationary_sma_20d_zscore,stationary_sma_20d_zscore_shift,stationary_sma_20d_zscore_shift_2,stationary_sma_20d_zscore_min,stationary_sma_20d_zscore_max,sma_50d,sma_50d_shift,sma_50d_coef,sma_50d_std,stationary_sma_50d,stationary_sma_50d_zscore,stationary_sma_50d_zscore_shift,stationary_sma_50d_zscore_shift_2,stationary_sma_50d_zscore_min,stationary_sma_50d_zscore_max,sma_100d,sma_100d_shift,sma_100d_coef,sma_100d_std,stationary_sma_100d,stationary_sma_100d_zscore,stationary_sma_100d_zscore_shift,stationary_sma_100d_zscore_shift_2,stationary_sma_100d_zscore_min,stationary_sma_100d_zscore_max,sma_200d,sma_200d_shift,sma_200d_coef,sma_200d_std,stationary_sma_200d,stationary_sma_200d_zscore,stationary_sma_200d_zscore_shift,stationary_sma_200d_zscore_shift_2,stationary_sma_200d_zscore_min,stationary_sma_200d_zscore_max,stationary_sma_10d_zscore_min_shift_1,stationary_sma_10d_zscore_min_shift_2,stationary_sma_10d_zscore_min_shift_3,stationary_sma_10d_zscore_min_all,stationary_sma_10d_zscore_min_all_shift,stationary_sma_10d_zscore_min_all_shift_2,stationary_sma_20d_zscore_min_shift_1,stationary_sma_20d_zscore_min_shift_2,stationary_sma_20d_zscore_min_shift_3,stationary_sma_20d_zscore_min_all,stationary_sma_20d_zscore_min_all_shift,stationary_sma_20d_zscore_min_all_shift_2,stationary_sma_100d_zscore_min_shift_1,stationary_sma_100d_zscore_min_shift_2,stationary_sma_100d_zscore_min_shift_3,stationary_sma_100d_zscore_min_all,stationary_sma_100d_zscore_min_all_shift,stationary_sma_100d_zscore_min_all_shift_2,stationary_sma_200d_zscore_min_shift_1,stationary_sma_200d_zscore_min_shift_2,stationary_sma_200d_zscore_min_shift_3,stationary_sma_200d_zscore_min_all,stationary_sma_200d_zscore_min_all_shift,stationary_sma_200d_zscore_min_all_shift_2,sma_10d_20d_ratio,sma_10d_20d_ratio_shift,sma_10d_20d_ratio_shift_2,sma_10d_20d_ratio_shift_3,sma_10d_20d_ratio_coef_2d,sma_10d_20d_ratio_avg,sma_10d_20d_ratio_std,sma_10d_20d_ratio_2std_up,sma_10d_20d_ratio_2std_down,sma_10d_20d_ratio_2std_up_diff,sma_10d_20d_ratio_2std_down_diff,sma_10d_20d_ratio_15std_up,sma_10d_20d_ratio_15std_down,sma_10d_20d_ratio_15std_up_diff,sma_10d_20d_ratio_15std_down_diff,sma_10d_20d_ratio_1std_up,sma_10d_20d_ratio_1std_down,sma_10d_20d_ratio_1std_up_diff,sma_10d_20d_ratio_1std_down_diff,sma_10d_50d_ratio,sma_10d_50d_ratio_shift,sma_10d_50d_ratio_shift_2,sma_10d_50d_ratio_shift_3,sma_10d_50d_ratio_coef_2d,sma_10d_50d_ratio_avg,sma_10d_50d_ratio_std,sma_10d_50d_ratio_2std_up,sma_10d_50d_ratio_2std_down,sma_10d_50d_ratio_2std_up_diff,sma_10d_50d_ratio_2std_down_diff,sma_10d_50d_ratio_15std_up,sma_10d_50d_ratio_15std_down,sma_10d_50d_ratio_15std_up_diff,sma_10d_50d_ratio_15std_down_diff,sma_10d_50d_ratio_1std_up,sma_10d_50d_ratio_1std_down,sma_10d_50d_ratio_1std_up_diff,sma_10d_50d_ratio_1std_down_diff,sma_10d_100d_ratio,sma_10d_100d_ratio_shift,sma_10d_100d_ratio_shift_2,sma_10d_100d_ratio_shift_3,sma_10d_100d_ratio_coef_2d,sma_10d_100d_ratio_avg,sma_10d_100d_ratio_std,sma_10d_100d_ratio_2std_up,sma_10d_100d_ratio_2std_down,sma_10d_100d_ratio_2std_up_diff,sma_10d_100d_ratio_2std_down_diff,sma_10d_100d_ratio_15std_up,sma_10d_100d_ra

## See Trades generated

In [6]:
def show_me_trades(no_of_days, buy_signal, clean_data):
    
    # Define min date
    min_date = clean_data['timestamp'].max() - timedelta(days=no_of_days)
    
    relevant_columns = ['timestamp', 'symbol','name', 'industry', 'next_7d_return', 'next_14d_return', 'next_21d_return']

    # Create Metrics
    metrics = clean_data[(clean_data[buy_signal] == True)][relevant_columns]
    metrics['year'] = pd.DatetimeIndex(metrics['timestamp']).year
    metrics_a = metrics.groupby(['symbol', 'name', 'industry', 'year']).count()['timestamp'].reset_index()
    metrics_b = metrics.groupby(['symbol', 'name', 'industry', 'year']).mean()[['next_7d_return', 'next_14d_return', 'next_21d_return']].reset_index()
    metrics = pd.merge(metrics_a, metrics_b, on=['symbol', 'name', 'industry', 'year'])

    # Create set of good symbols
    list_of_good = set(metrics[(metrics['next_14d_return'] > 1) & (metrics['next_21d_return'] > 1) & (metrics['year'] >= 2019)]['symbol'])

    # Run for the past 5 days
    print('Only with past record')
    display(clean_data[(clean_data['timestamp'] > min_date) & (clean_data[buy_signal] == True) & (clean_data['symbol'].isin(list_of_good))][['timestamp', 'symbol', 'next_7d_return', 'next_14d_return', 'next_21d_return', 'close_price_x']])
    print('Everything')
    display(clean_data[(clean_data['timestamp'] > min_date) & (clean_data[buy_signal] == True)][['timestamp', 'symbol', 'next_7d_return', 'next_14d_return', 'next_21d_return']])

    # Last days buys
    last_days_symbol = set(clean_data[(clean_data['timestamp'] > min_date) & (clean_data[buy_signal] == True) & (clean_data['symbol'].isin(list_of_good))]['symbol'])
    display(metrics[metrics['symbol'].isin(last_days_symbol)].sort_values('symbol'))

In [7]:
show_me_trades(2, 'buy_signal?1', trades)


Only with past record


,timestamp,symbol,next_7d_return,next_14d_return,next_21d_return,close_price_x
838,2020-05-26 13:30:00,CSGS,NaN,NaN,NaN,47.169998
838,2020-05-26 13:30:00,ATEC,NaN,NaN,NaN,4.600000
838,2020-05-26 13:30:00,CUBI,NaN,NaN,NaN,10.910000


Everything


,timestamp,symbol,next_7d_return,next_14d_return,next_21d_return
838,2020-05-26 13:30:00,CSGS,NaN,NaN,NaN
838,2020-05-26 13:30:00,ATEC,NaN,NaN,NaN
838,2020-05-26 13:30:00,CUBI,NaN,NaN,NaN


,symbol,name,industry,year,timestamp,next_7d_return,next_14d_return,next_21d_return
11,ATEC,"Alphatec Holdings, Inc.",Healthcare,2017,15,0.983573,1.030661,1.071332
12,ATEC,"Alphatec Holdings, Inc.",Healthcare,2018,5,0.999368,1.045615,1.071569
13,ATEC,"Alphatec Holdings, Inc.",Healthcare,2019,2,1.252517,1.711694,1.881271
14,ATEC,"Alphatec Holdings, Inc.",Healthcare,2020,6,0.990071,0.999031,0.978621
56,CSGS,"CSG Systems International, Inc.",Technology,2017,13,0.983070,0.981116,0.988738
57,CSGS,"CSG Systems International, Inc.",Technology,2018,10,1.026576,1.034342,1.006891
58,CSGS,"CSG Systems International, Inc.",Technology,2019,2,1.046276,1.094179,1.127985
59,CSGS,"CSG Systems International, Inc.",Technology,2020,6,1.000957,1.050657,1.063398
65,CUBI,"Customers Bancorp, Inc.",Financial,2017,14,1.013013,1.003525,1.021757
66,CUBI,"Customers Bancorp, Inc.",Financial,2018,16,0.967230,0.945721,0.949877
